In [2]:
import os
import re
import numpy as np
import json
from nltk.tokenize import word_tokenize, sent_tokenize
import seaborn as sns

In [40]:
DATA = "data/"
files = os.listdir(DATA)

In [41]:
texts = [open(DATA + x).read() for x in files]

In [4]:
for t in files:
    if t.endswith("~"):
        os.rename(f"{DATA}{t}", f"{DATA}_{t[:-1]}")

In [48]:
texts = [open("broken.txt").read()]

In [49]:
#remove all <p> tags
texts = list(map(lambda x: re.sub("<p>", "", x), texts))
#remove references, i.e. [1]
texts = list(map(lambda x: re.sub("\[[0-9, - c.]+\]", "", x), texts))
#remove headers
#texts = list(map(lambda x: re.sub("\<h\>[\w -«»:–]+", "", x), texts)) 
# replace some characters
texts = list(map(lambda x: re.sub("ѐ", "e", x), texts))
# remove <li> tag
texts = list(map(lambda x: x.replace("\n<li>", " "), texts))
# replace &quot;
texts = list(map(lambda x: x.replace("&quot;", "\""), texts))

texts = list(map(lambda x: x.replace("<keys>", "<key>"), texts))



In [43]:

paragrap_annotation = []


In [50]:
broken_texts = []
header_id = set()
for i, papers in enumerate(texts):
    for j,sample in enumerate(papers.split("\n\n")):
        new_text_annot = {"header": '', "abstract": '', "keys": ''}
        try:
            new_text_annot['header'] = re.findall("<h>(.+)\\n", sample)[0]
        except IndexError:
            broken_texts.append(sample)
            continue
        if new_text_annot['header'] in header_id:
            continue
        else:
            header_id.add(new_text_annot['header'])
        try:
            new_text_annot['abstract'] = re.findall("<abs>(.+)\\n", sample)[0]
        except IndexError:
            print(f"Not found abstract in {i}, {j}")
        try:
            
            new_text_annot['keys'] = re.findall("<key>(.+)\\n", sample)[0].replace("Ключевые слова: ", "")
        except IndexError:
            print(f"Not found keys in {i}, {j}")
        paragraphs = re.findall("<pa>(.+)\\n", sample)
        if any([True if "\n" in x else False for x in paragraphs]):
            raise ValueError
        paragraphs = '\n'.join(paragraphs)
        new_text_annot["indices"] = [i for i, c in enumerate(paragraphs) if c == "\n"]
        new_text_annot["text"] = paragraphs.replace("\n", " ")
        paragrap_annotation.append(new_text_annot)
        

In [51]:
len(broken_texts)

0

In [47]:
with open("broken.txt", 'w') as f:
    f.write("\n\n".join(broken_texts))

['Ключевые слова: затраты на рабочую силу, затраты на оплату труда, затраты на заработную плату']

In [55]:

with open("paragraph_corpus.json", 'w') as f:
    json.dump(paragrap_annotation, f)

In [5]:
jtexts = ' '.join(texts)
jtexts = jtexts.replace("\n", '')

In [ ]:
#segments = jtexts.split("<pa>")
#len(segments)

In [8]:
articles = jtexts.split("<h>")

In [16]:
art_seg = list(map(lambda x: x.split("<pa>"), articles))

# Loading

In [3]:

with open("paragraph_corpus.json",) as f:
    corpus = json.load( f)

In [24]:
def visualize(text, enter_idx):
    c = 0
    prev = -1
    new_text = ''
    for i in enter_idx:
        new_text += text[prev+1:i]
        new_text += "\n"
        #c += 1
        prev = i# + c
    return new_text

In [28]:
texts = [c["text"] for c in corpus]

In [31]:
with open("corpus.txt", 'w') as f:
    f.write("\n\n".join(texts))

In [8]:
text[:526]

'Мед, цветочная пыльца (обножка), маточное молочко, прополис – уникальные  продукты природы, содержащие в сбалансированном сочетании ряд важнейших биологически активных компонентов: белков, липидов, аминокислот, витаминов, ферментов, макро- и микроэлементов. Обладая достаточно высокой терапевтической активностью, они действуют мягче, физиологичнее синтетических лекарственных средств, характеризуются высокой степью безопасности, а относительная дешевизна делает их доступными практически для всех социальных слоев населения. Пыльца, например, является прекрасным лечебным и профилактическим средством при многих заболеваниях, а также биостимулятором, благотворно действующим на организм. Пыльца содержит все микроэлементы и аминокислоты, необходимые для нормального развития организма. Особенно много в ней калия, железа, меди и кобальта. Провитамина А (каротина) в пыльце в 20 раз больше, чем в моркови. В ней есть витамины группы В, Е, С, D, Р, РР, К и др., фитогормоны, стимулирующие рост ткане

In [32]:
new_annot = []
for i in range(len(corpus)):
    a = corpus[i]
    idx = a.pop("indices")
    a.pop("text")
    a["carriage_idx"] = idx
    a["text_id"] = i
    new_annot.append(a)

In [35]:

with open("annotation.json", 'w') as f:
    json.dump(new_annot, f)

In [ ]:
art_seg_sent = list(map(lambda x: [sent_tokenize(y) for y in x[3:]], art_seg))
art_seg_sent.pop(0)